In [1]:
import numpy as np

import pandas as pd
import os

In [2]:
tasks = ['TCGA-LUAD_survival', 'TCGA-LUAD_survival-VIM', 'TCGA-LUAD_survival-VMAMBA']
for task in tasks:
    log_dir = os.path.join('logs/configs', task)
    rng = range(5)

    versions = os.listdir(log_dir)
    for version in versions: # each method
        if os.path.isfile(os.path.join(log_dir, version)):
            continue
        seed_rng = [1]
        for seed in seed_rng:
            c_index_list = []        
            final_csv_path = os.path.join(log_dir, version, f's{seed}', 'result.csv')
            f = open(final_csv_path, 'w')
            f.write('fold,c_index\n')
            for fold in rng:
                result_txt_path = os.path.join(log_dir, version, f's{seed}', f'fold{fold}', 'result.txt')
                if not os.path.exists(result_txt_path):
                    continue
                result_str = open(result_txt_path, 'r').read()
                c_index = float(result_str.split('\n')[0].split(' ')[-1])

                f.write(f'{fold},{c_index:.4f}\n')

                c_index_list.append(c_index)

            mean_acc, std_acc = np.mean(c_index_list), np.std(c_index_list)
        
            f.write(f'mean±std,{mean_acc:.4f}±{std_acc:.4f}\n')
            f.close()
            

In [3]:
for task in tasks:
    log_dir = os.path.join('logs/configs', task)

    compare_result = open(os.path.join(log_dir, 'compare_result.csv'), 'w')
    compare_result.write('method,c-index\n')

    versions = os.listdir(log_dir)
    for version in versions: # each method
        final_csv = os.path.join(log_dir, version, f's1/result.csv')
        if not os.path.exists(final_csv): continue

        with open(final_csv, 'r') as f:
            lines = f.readlines()
            result = lines[-1].split(',')
        result[0] = version
        result = ','.join(result)
        compare_result.write(result)
    compare_result.close()